Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

Data Preprocessing

In [17]:
# data collection ->

df = pd.read_csv("../data/raw/heart_disease_encoded.csv")

# 0. removing un-necessary features -> 

df = df.drop(columns=['id']) # id does not carry any meaningful information for model training

In [ ]:
# 1. HANDLING MISSING VALUES(NaN) ->

df.isna().sum() # o/p -> number of missing values for each column
# result : the following columns have missing values -> 
# trestbps, chol, fbs, restecg, thalch, exang, oldpeak, slope, ca, thal

missing_features = ['trestbps','chol','fbs','restecg','thalch','exang','oldpeak','slope','ca','thal']
categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'target']
continuous_features = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']


# CONTEXT -> in the images directory you can check in the univariate analysis folder that non of the
#            features follow normal distribution. 
# INFERENCE -> median -> measure of central tendency for continuous features
#              mode -> measure of central tendency for categorical features

# function to handle missing values ->
def fill_missing_values(df,categorical_f,continuous_f):
    for feature in df.columns:
        if(feature in categorical_f):
            df[feature] = df[feature].fillna(value=df[feature].mode()[0])
        elif(feature in continuous_f) :
            df[feature] = df[feature].fillna(value=df[feature].median())

fill_missing_values(df,categorical_features,continuous_features)

df.isna().sum()

In [ ]:
# 2. DROPING ANY DUPLICATE ROWS IN THE df ->
print(df.shape)
# originally -> there were 920 rows
df = df.drop_duplicates()
print(df.shape)
# current -> there are 918 rows -> 2 duplicate rows are dropped

In [ ]:
# 3. 